In [ ]:
import requests
import pandas as pd

headers = {
    'Content-Type': 'application/json'
}
requestResponse = requests.get("https://api.tiingo.com/iex/aapl/prices?startDate=2019-01-02&resampleFreq=5min&columns=open,high,low,close,volume&token=1bb8adbd31a937981ecc6d299b941b788e0ef3d8", headers=headers)
print(requestResponse.json())

In [ ]:
pd.DataFrame(requestResponse.json()).tail()

In [ ]:
headers = {
    'Content-Type': 'application/json'
}
requestResponse = requests.get("https://api.tiingo.com/tiingo/news?token=1bb8adbd31a937981ecc6d299b941b788e0ef3d8", headers=headers)
print(requestResponse.json())

In [ ]:
news_df = pd.DataFrame(requestResponse.json())

In [ ]:
news_df

In [ ]:
import requests
import pandas as pd
import time

# 1. Setup: The SEC requires a User-Agent string with your email (for administrative contact)
headers = {'User-Agent': 'JustinNewberry (jstnnwbrr@gmail.com)'}

def get_sec_data():
    print("Fetching Master Ticker List from SEC...")
    # Get the list of all tickers (CIK, Ticker, Title)
    tickers_url = "https://www.sec.gov/files/company_tickers.json"
    response = requests.get(tickers_url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error fetching list: {response.status_code}")
        return
    
    # Parse the JSON into a DataFrame
    data = response.json()
    company_list = [value for key, value in data.items()]
    df = pd.DataFrame(company_list)
        
    # 2. Create a DataFrame
    df = pd.DataFrame(company_list)
    df.rename(columns={'cik_str': 'CIK', 'ticker': 'Ticker', 'title': 'Company'}, inplace=True)
    
    print(f"Found {len(df)} companies. Fetching Industry data...")
    
    # 3. Fetch Industry (SIC Code) for each company
    # It will take about 15-20 minutes to run for all companies due to rate limits.
    
    industries = []
    sic_codes = []
    
    for cik in df['CIK']: 
        # Pad CIK with zeros to 10 digits
        padded_cik = str(cik).zfill(10)
        url = f"https://data.sec.gov/submissions/CIK{padded_cik}.json"
        
        try:
            r = requests.get(url, headers=headers)
            if r.status_code == 200:
                company_data = r.json()
                industries.append(company_data.get('sicDescription', 'N/A'))
                sic_codes.append(company_data.get('sic', 'N/A'))
            else:
                industries.append("Error")
                sic_codes.append("Error")
        except Exception as e:
            industries.append("Error")
            sic_codes.append("Error")
            
        # Sleep to respect SEC rate limit (max 10 requests/sec)
        time.sleep(0.11) 

    # Add columns to DataFrame
    df_final = df.copy()
    df_final['SIC Code'] = sic_codes
    df_final['Industry (SEC)'] = industries
    
    # Display the result
    print(df_final)
    
    # Export to CSV
    df_final.to_csv("C:\\Users\\sar81\\Desktop\\sec_companies_list.csv", index=False)
    print("Saved to sec_companies_list.csv")

get_sec_data()